# Ray getting started

* [Programming in Ray: Tips for first-time users](https://rise.cs.berkeley.edu/blog/ray-tips-for-first-time-users/)
* [Modern Parallel and Distributed Python: A Quick Tutorial on Ray](https://towardsdatascience.com/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray-99f8d70369b8)

# Installation

There is no conda package and "ray" in Bioconda channel is for genom analysis.

In [1]:
!pip install -q ray


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import re
import ray

# Use Ray

## Start Ray on a sngle machine

* [ray.init](https://docs.ray.io/en/latest/package-ref.html#ray-init)

```ray.init()``` will start Ray and it detects available resource so as to utilize all cores on the machine. 

In [3]:
# ray.init()
ray.init(num_cpus=4, num_gpus=0)

2023-04-26 07:31:18,627	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.13
Ray version:,2.3.0


## Connect to a cluster

Need first run ray start on the command line to start the Ray cluster services. Then connect to an existing cluster. See [Ray Cluster Overview](https://docs.ray.io/en/master/cluster/index.html)

```ray.init(address=<cluster-address>)```

---

# Remote execution

To create a proxy instance to remote-call a Python function.

* [ray.remote](https://docs.ray.io/en/latest/package-ref.html#ray-remote)
               
> This can be used with no arguments to define a remote function or actor

### Via creating a proxy

In [4]:
def f(x):
    return x * x

remote_f = ray.remote(
    f
)
future_f = remote_f.remote(4)
print(ray.get(future_f))

16


### Via the decorator

In [5]:
@ray.remote
def g(x):
    return x * x
future_g = g.remote(4)
print(ray.get(future_g))

16


# Using object store via put/get

Do not passing large data over the network every time calling remote functions. Insteaad, store the data in shared object store and reference it from the remote functions.

* [Put and Get](https://docs.ray.io/en/stable/tutorial.html#put-and-get)

> ```ray.put(object)``` takes a Python object and **copies it to the Ray distributed shared-memory object store** in the node(s). The object is **immutable**.
> 
> ```ray.put(object)``` returns a reference which identifies the now remote-able object. If we save it in a variable ```ref``` e.g ```ref = ray.put(x)```, remote functions can take it as its argument and operate on the corresponding **acutual** remote object. **NO NEED to ```ray.get(ref)``` in the remote function** to use the object instance.
> 
> For objects like arrays, we can use shared memory and avoid copying the object. If the remote object does not live on the node where the worker calls ray.get(ref), then the remote object will be transferred first to the node.


* [Objects](https://docs.ray.io/en/latest/ray-core/objects.html)

> You can use the ray.get() method to fetch the result of a remote object from an object ref. If the current node’s object store does not contain the object, the object is downloaded.

* [How exactly does Ray share data to workers?](https://stackoverflow.com/questions/58082023/how-exactly-does-ray-share-data-to-workers)


In [6]:
@ray.remote(num_returns=1)
def runner(instances): # <--- caller passes references 
    import re
    
    result = []
    for obj in instances: # <--- Just use the object without executing ray.get().
        match = re.search(r"http://(.*).com", obj, re.IGNORECASE)
        result.append(match.group(1))
        
    return result

In [7]:
urls = [
    "http://gmail.com",
    "http://facebook.com"
]
reference_to_urls = ray.put(urls)

ray.get(runner.remote(reference_to_urls))  # Passing the object reference

['gmail', 'facebook']

### Avoding ray.get() in the remote function

* [Objects](https://docs.ray.io/en/latest/ray-core/objects.html)

When an ray object is passed within a Python wrapper object e.g. list or dictionary, you need to call ```ray.get(wrapper)``` to de-reference (de-serialize in the node) the wrapper to access the ray object.


In [8]:
@ray.remote(num_returns=1)
def func_to_handle_wrapped(wrapper): # <--- caller passes references 
    import re

    # Explicitly de-reference the wrapper.
    # We would like keep the remote function as pure simple python funciton 
    # without introducing ray specific handling.
    instances = ray.get(wrapper)
    
    result = []
    for obj in instances: # <--- Just use the object without executing ray.get().
        match = re.search(r"http://(.*).com", obj, re.IGNORECASE)
        result.append(match.group(1))
        
    return result

In [9]:
wrapped = [
    ray.put("http://gmail.com"),
    ray.put("http://facebook.com")
]

ray.get(func_to_handle_wrapped.remote(wrapped))  # Passing the object reference

['gmail', 'facebook']

**Avoid wrapping** that requires ```ray.get(wrapper)``` inside the remote function by always pass a ray object only, and stick to returning single value by ```@ray.remote(num_returns=1)```.

In [10]:
@ray.remote(num_returns=1)
def func_NOT_to_handle_wrapped(instances): # <--- caller passes references 
    import re

    # No de-referencing with ray.get() in the remote function.
    # We would like keep the remote function as pure simple python funciton 
    # instances = ray.get(wrapper)
    
    result = []
    for obj in instances: # <--- Just use the object without executing ray.get().
        match = re.search(r"http://(.*).com", obj, re.IGNORECASE)
        result.append(match.group(1))
        
    return result

In [11]:
urls = [
    "http://gmail.com",
    "http://facebook.com"
]
# Create a ray object from the entire python object
# Maintain one-to-one relation between (ray-object, python-object)
instances = ray.put(urls)

# Pass only ray object as remote function argument
ray.get(func_NOT_to_handle_wrapped.remote(instances))

['gmail', 'facebook']

## Stop/disconnect

In [12]:
ray.shutdown()